In [336]:
import findspark

In [337]:
findspark.init()

In [338]:
import pyspark

In [7]:
pyspark.__version__

'3.5.5'

In [342]:
from pyspark.sql import SparkSession

In [343]:
spark = SparkSession.builder.appName("practice").getOrCreate()

25/05/13 08:56:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [344]:
spark.sparkContext

<SparkContext master=yarn appName=ch06>

In [349]:
data = [
    "Fri Aug 25 13:55:02 -0700 2017",
    "Sun Jul 30 07:44:10 -0700 2017",
    "Mon Jul 24 02:48:17 -0700 2017",
    "Mon Jul 24 02:33:09 -0700 2017",
    "Mon Jul 24 02:28:14 -0700 2017"
]

data = map(lambda x: (x,), data)

df = spark.createDataFrame(data, ["date_added"])


In [350]:
df.show(2, False)

+------------------------------+
|date_added                    |
+------------------------------+
|Fri Aug 25 13:55:02 -0700 2017|
|Sun Jul 30 07:44:10 -0700 2017|
+------------------------------+
only showing top 2 rows



In [354]:
from pyspark.sql.functions import to_timestamp, unix_timestamp

format_str = "EEE MMM dd HH:mm:ss XX yyyy"

df.withColumn("date_added", unix_timestamp("date_added", format_str)).show()

SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.DATETIME_PATTERN_RECOGNITION] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to recognize 'EEE MMM dd HH:mm:ss XX yyyy' pattern in the DateTimeFormatter. 1) You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0. 2) You can form a valid datetime pattern with the guide from 'https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html'.

In [355]:
import datetime

In [361]:
str1 = "Mon Jul 24 02:28:14 -0700 2017"
format_str = "%a %b %d %H:%M:%S %z %Y"
datetime.datetime.strptime(str1, format_str)

datetime.datetime(2017, 7, 24, 2, 28, 14, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200)))

In [374]:
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType
import datetime

data = [
    "Fri Aug 25 13:55:02 -0700 2017",
    "Sun Jul 30 07:44:10 -0700 2017",
    "Mon Jul 24 02:48:17 -0700 2017",
    "Mon Jul 24 02:33:09 -0700 2017",
    "Mon Jul 24 02:28:14 -0700 2017"
]

data = map(lambda x: (x,), data)

df = spark.createDataFrame(data, ["date_added"])

# Define the parsing function using Python's strptime
def parse_date(date_str):
    try:
        return datetime.datetime.strptime(date_str, "%a %b %d %H:%M:%S %z %Y")
    except:
        return None

# Register as UDF
parse_date_udf = udf(parse_date, TimestampType())

# Use on your DataFrame
df1 = df.withColumn("parsed_date", parse_date_udf("date_added"))
df1.show(5, False)

+------------------------------+-------------------+
|date_added                    |parsed_date        |
+------------------------------+-------------------+
|Fri Aug 25 13:55:02 -0700 2017|2017-08-25 20:55:02|
|Sun Jul 30 07:44:10 -0700 2017|2017-07-30 14:44:10|
|Mon Jul 24 02:48:17 -0700 2017|2017-07-24 09:48:17|
|Mon Jul 24 02:33:09 -0700 2017|2017-07-24 09:33:09|
|Mon Jul 24 02:28:14 -0700 2017|2017-07-24 09:28:14|
+------------------------------+-------------------+

